# Paddle OCR을 실행해보자

In [ ]:
# CUDA-11.8이 설치되어 있는 GPU 사용 가능한 컴퓨터기 때문에 gpu 버전 다운로드
# !pip install paddlepaddle-gpu
%pip install paddlepaddle-gpu

In [ ]:
# PaddleOCR whl package 설치
# !pip install "paddleocr>=2.0.1"
%pip install paddleocr

# 한번 사용해보도록 하자

실행하기 전, 컴퓨터에 cuDNN이 설치되어있는지 확인하자. 없다면 설치

CUDA 11.8 사용 기준으로

wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
sudo dpkg -i cuda-keyring_1.1-1_all.deb
sudo apt-get update
sudo apt-get -y install cudnn
sudo apt-get -y install cudnn-cuda-11

In [ ]:
import paddle
print(f"Is compiled with CUDA? {paddle.is_compiled_with_cuda()}")
print(f"CUDA devices:, {paddle.device.cuda.device_count()}")

In [ ]:
# 콘솔에 아래 명령 입력
# paddleocr --image_dir .책표지_기타_000001.jpg --lang=korean

In [ ]:
from paddleocr import PaddleOCR,draw_ocr

ocr = PaddleOCR(lang="korean")

In [ ]:
# 코드로 실행하자면?

img_path = 'test_imgs/test33.jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)


# draw result
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path='MaruBuri-Regular.ttf')
im_show = Image.fromarray(im_show)
im_show.save('first_ocr_result.jpg')
im_show.show()

In [ ]:
# for idx in range(len(boxes)):
#     print(f"boxes: {boxes[idx]}, txts: {txts[idx]}")

# 한글 또는 영어, 숫자 txt만 포함된 box로 재구성
boxes = [boxes[idx] for idx in range(len(boxes)) if txts[idx].isalnum()]

for idx in range(len(boxes)):
    print(f"boxes: {boxes[idx]}")

In [ ]:
# !pip install --upgrade pip
%pip install --upgrade pip

In [ ]:
# !pip install opencv-python
%pip install opencv-python

In [ ]:
import cv2
import numpy as np

# 1. cv2 imread로 그레이스케일 이미지 획득
image = cv2.imread('test_imgs/test33.jpg', cv2.IMREAD_GRAYSCALE)

# 2. 연산 효율성을 올리기 위해 boxes를 numpy array로 변환
boxes_np_arr = np.array([])
if boxes and len(boxes) > 0:
    boxes_np_arr = np.array([np.array(box) for box in boxes])

    for box in boxes:
        print(box)
else:
    print("No boxes")

In [ ]:
# 3. 각 box = bounding box
# bounding box 영역 안에 있는 각 영역을 Crop해서 새로운 이미지로 추출한다
import os

current_path = os.getcwd()
os.makedirs(current_path + "/cropped", exist_ok=True)
croped_images = []

for idx, box in enumerate(boxes_np_arr):
    # box 점 정렬
    rect = np.zeros((4, 2), dtype="float32")
    s = box.sum(axis=1) # 네 좌표의 합
    diff = np.diff(box, axis=1) # 네 좌표의 차이
    rect[0] = box[np.argmin(s)] # 좌상단 좌표
    rect[2] = box[np.argmax(s)] # 우하단 좌표
    rect[1] = box[np.argmin(diff)] # 우상단 좌표
    rect[3] = box[np.argmax(diff)] # 좌하단 좌표

    # rect = [ 좌상, 우상, 우하, 좌하 ]

    # 네 꼭짓점 좌표를 numpy array로 변환
    # 이때, 검출 영역이 너무 Fit하게 되어있어서 +10을 해준다
    offset = 10
    image_height, image_width = image.shape[:2]
    image_height -= 1
    image_width -= 1

    # rect[0] = 좌상단
    x1 = max(0, rect[0][0] - offset)
    y1 = max(0, rect[0][1] - offset)
    # rect[1] = 우상단
    x2 = min(rect[1][0] + offset, image_width)
    y2 = max(0, rect[1][1] - offset)
    # rect[2] = 우하단
    x3 = min(rect[2][0] + offset, image_width)
    y3 = min(rect[2][1] + offset, image_height)
    # rect[3] = 좌하단
    x4 = max(0, rect[3][0] - offset)
    y4 = min(rect[3][1] + offset, image_height)
    pts = np.array(
        [
            [x1, y1],
            [x2, y2],
            [x3, y3],
            [x4, y4]
        ],
        dtype="float32"
    )

    # 네 좌표를 바탕으로 이미지의 너비 및 높이 계산
    widthA = np.sqrt((pts[2][0] - pts[3][0])**2 + (pts[2][1] - pts[3][1])**2)
    widthB = np.sqrt((pts[1][0] - pts[0][0])**2 + (pts[1][1] - pts[0][1])**2)
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt((pts[1][0] - pts[2][0])**2 + (pts[1][1] - pts[2][1])**2)
    heightB = np.sqrt((pts[0][0] - pts[3][0])**2 + (pts[0][1] - pts[3][1])**2)
    maxHeight = max(int(heightA), int(heightB))

    # print(maxWidth, maxHeight)

    # 출력될 이미지 좌표 설정
    dst = np.array(
        [
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ],
        dtype="float32"
    )

    # Perspective Transformation
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # 음영 제거 -> 모폴로지 연산
    kernel_size = 150
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

    background = cv2.morphologyEx(warped, cv2.MORPH_OPEN, kernel)
    gray_minus_bg = cv2.subtract(warped, background)
    normed = cv2.normalize(gray_minus_bg, None, 0, 255, cv2.NORM_MINMAX)

    croped_images.append(normed)
    cv2.imwrite(f"./cropped/croped_{idx}.jpg", normed)

In [ ]:
print(len(croped_images))

In [ ]:
# 이미지 전처리
os.makedirs(current_path + "/preprocessed", exist_ok=True)
cropped_dir = current_path + "/cropped"
for idx, filename in enumerate(os.listdir(cropped_dir)):
    croped_image = cv2.imread(f"{cropped_dir}/{filename}", cv2.IMREAD_GRAYSCALE)
    
    # 이진화
    _, bin = cv2.threshold(croped_image, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 가우시안 블러
    blurred = cv2.GaussianBlur(bin, (5, 5), 0)

    # 만약 배경이 반전되어 있다면 invert
    if np.mean(blurred) < 128:
        blurred = cv2.bitwise_not(blurred)
    
    # 이미지 샤프닝
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened = cv2.filter2D(blurred, -1, kernel)
    
    # 침식 연산
    kernel_size = 3
    closed = cv2.erode(sharpened, np.ones((kernel_size, kernel_size), np.uint8), iterations=1)

    cv2.imwrite(f"./preprocessed/preprocessed_{idx}.png", closed)


In [ ]:
# 전처리 이미지 불러와서 테두리 따고, 한 글자씩 추출
binarizationed_images = []
preprocessed_dir = current_path + "/preprocessed/"

for filename in sorted(os.listdir(preprocessed_dir)):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Correct way to check multiple extensions
        img_path = os.path.join(preprocessed_dir, filename)
        # Load as grayscale directly
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresh_inv = cv2.bitwise_not(thresh)

        if thresh_inv is not None:
            binarizationed_images.append(thresh_inv)

print(f"Loaded {len(binarizationed_images)} images")


In [ ]:
os.makedirs(current_path + "/contours", exist_ok=True)

for idx, image in enumerate(binarizationed_images):
    contours, _ = cv2.findContours(
        image,
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )

    # 외곽선 기반으로 각 글자 이미지 추출
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)

        print(f"Contour: {x}, {y}, {w}, {h}")
        
        # 면적이나 가로세로 비율을 엄격하게 조정
        area = cv2.contourArea(contour)
        if aspect_ratio > 1.2 or aspect_ratio < 0.5 or area < 10:
            continue

        # w, h 비율이 1.4 : 1 이상 넘어가는 이미지는 제거
        # if w / h > 1.4:
        #     continue

        character = image[y:y+h, x:x+w]
        character = ~character
        
        # 상하좌우 5px 흰색배경 추가
        padding = 5
        expanded = cv2.copyMakeBorder(
            character, 
            padding, padding, padding, padding, 
            cv2.BORDER_CONSTANT, 
            value=(255, 255, 255)
        )

        cv2.imwrite(f"./contours/contour_{idx}_{x}_{y}_{w}_{h}.jpg", expanded)

In [ ]:
# 이미지 절대 크기가 30px 이상인 이미지만 선택
os.makedirs(current_path + "/best_size", exist_ok=True)
contours_dir = current_path + "/contours"
for idx, filename in enumerate(os.listdir(contours_dir)):
    character = cv2.imread(f"{contours_dir}/{filename}", cv2.IMREAD_GRAYSCALE)
    height, width = character.shape[:2]

    if height < 30 or width < 30:
        continue
    
    cv2.imwrite(f"./best_size/character_{idx}.jpg", character)


In [ ]:
import re

os.makedirs(current_path + "/final", exist_ok=True)
save_dir = current_path + "/final/"
best_size_dir = current_path + "/best_size/"

for idx, filename in enumerate(sorted(os.listdir(best_size_dir))):
    if filename.endswith((".png", ".jpg", ".jpeg")):
        img = cv2.imread(f"{best_size_dir}" + filename, cv2.IMREAD_GRAYSCALE)
        result = ocr.ocr(img, det=False, rec=True, cls=False)

        # 전체 픽셀 중 흰색 픽셀 대 검정 픽셀 비율이 80% 이상인 경우 continue
        if np.mean(img) > 200:
            continue

        if not result or not result[0] or not result[0][0]: # 인식 못하면 continue
            continue

        text = result[0][0][0]
        confidence = result[0][0][1]

        if re.search(r"[^가-힣ㄱ-ㅎㅏ-ㅣ]", text): # 한글이 아닌 문자가 있는 경우 continue
            continue

        if text and len(text) == 1 and confidence > 0.8:
            print(f"#### {idx}th // Character: {text}, Confidence: {confidence}")
            cv2.imwrite(save_dir + f"{text}.jpg", img)


In [ ]:
# import shutil
# import os

# current_path = os.getcwd()

# shutil.rmtree(current_path + "/cropped")
# shutil.rmtree(current_path + "/preprocessed")
# shutil.rmtree(current_path + "/contours")
# shutil.rmtree(current_path + "/best_size")
# shutil.rmtree(current_path + "/final")

# print("All directories removed successfully")